In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-mnsforest')

In [3]:
aoi = ee.FeatureCollection('projects/ee-mnsforest/assets/roi_mamminasata')

In [4]:
def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_shadow = qa.bitwiseAnd(1 << 3).eq(0)
    cloud = qa.bitwiseAnd(1 << 5).eq(0)
    return image.updateMask(cloud.And(cloud_shadow))

In [5]:
def apply_scale_factors(image):
    optical = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical, None, True).addBands(thermal, None, True)

In [16]:
dataset = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2005-01-01', '2005-12-31') \
    .map(mask_clouds) \
    .map(apply_scale_factors)

In [17]:
composite = dataset.median().clip(aoi)

In [18]:
ndvi = composite.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
evi = composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {'NIR': composite.select('SR_B4'), 'RED': composite.select('SR_B3'), 'BLUE': composite.select('SR_B1')}
).rename('EVI')
savi = composite.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {'NIR': composite.select('SR_B4'), 'RED': composite.select('SR_B3')}
).rename('SAVI')
ndwi = composite.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI')
ndmi = composite.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDMI')
ndbi = composite.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDBI')
bsi = composite.expression(
    '((SWIR + RED) - (NIR + BLUE)) / ((SWIR + RED) + (NIR + BLUE))',
    {
        'SWIR': composite.select('SR_B5'),
        'RED': composite.select('SR_B3'),
        'NIR': composite.select('SR_B4'),
        'BLUE': composite.select('SR_B1')
    }
).rename('BSI')
mndwi = composite.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')

In [19]:
rgb_vis = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.05,
    'max': 0.4
}

In [20]:
Map = geemap.Map()
Map.centerObject(aoi, 9)
Map.addLayer(composite, rgb_vis, 'Landsat 7 RGB')
Map

Map(center=[-5.211515044202248, 119.61581530927582], controls=(WidgetControl(options=['position', 'transparent…

In [21]:
sr_bands = composite.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).toFloat()

In [22]:
bands_only = sr_bands
task1 = ee.batch.Export.image.toDrive(
    image=bands_only,
    description='Export_L7_SR_Bands',
    folder='GEE Landsat 7',
    fileNamePrefix='L7_2005_SR_Bands',
    scale=30,
    region=aoi.geometry(),
    fileFormat='GeoTIFF'
)
task1.start()

In [23]:
def export_index(image, name):
    task = ee.batch.Export.image.toDrive(
        image=image.toFloat(),
        description=f'Export_{name}',
        folder='GEE Landsat 7',
        fileNamePrefix=f'L7_2005_{name}',
        scale=30,
        region=aoi.geometry(),
        fileFormat='GeoTIFF'
    )
    task.start()

In [24]:
export_index(ndvi, 'NDVI')
export_index(evi, 'EVI')
export_index(savi, 'SAVI')
export_index(ndwi, 'NDWI')
export_index(ndmi, 'NDMI')
export_index(ndbi, 'NDBI')
export_index(bsi, 'BSI')
export_index(mndwi, 'MNDWI')